# Solver debugging capabilities

When setting up a model or a use case, it's pretty often handy to be able to introspect the solver (e.g. plot intermediate resolution steps).

In [ ]:
from cosapp.recorders import DataFrameRecorder
from cosapp.utils import set_log, LogLevel

from cpu.systems import CPUSystem
from cpu.utils.solver_trace import plot_solver_trace

cpu = CPUSystem("cpu")
set_log()

In [ ]:
from cosapp.drivers import NonLinearSolver

design = cpu.add_driver(NonLinearSolver('solver', factor=0.1, history=True))
design.extend(cpu.design_methods["exchanger_surface"])
design.runner.set_values({"fan.T_air": 40., "T_cpu": 80., "cpu.usage": 100.})

rec = design.runner.add_recorder(DataFrameRecorder(includes=['T_cpu', 'cpu.power', 'cpu.usage', 'cpu.heat_flow', 'exchanger.heat_flow', 'exchanger.h', 
                                                             'exchanger.surface', 'fan.tension', 'fan.fl_out.mass_flow', 'controler.T', 'controler.tension'], hold=False))
# run design
cpu.run_drivers()

In [ ]:
rec.data

In [ ]:
plot_solver_trace(design)

In [ ]:
from cosapp.systems import System


class ParabolicSystem(System):

    def setup(self):
        self.add_inward("a", 2.)
        self.add_inward("x", 1.)
        
        self.add_outward("y")

    def compute(self):
        self.y = self.a * self.x**2


p = ParabolicSystem("p")

p_design = p.add_driver(NonLinearSolver('solver', factor=1., history=True))
p_design.add_equation("y == 10.").add_unknown("x")
p.run_drivers()

plot_solver_trace(p_design)